In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import torch
from torch.nn import BCELoss, CrossEntropyLoss

import sys
sys.path.insert(0,'..')
from src.lstm import RPS_LSTM

DATA_PATH = '../data/'
import torch.nn as nn

In [ ]:
rnn = nn.LSTM(10, 20, 2)
input = torch.randn(5, 3, 10)
h0 = torch.randn(2, 3, 20)
c0 = torch.randn(2, 3, 20)
output, (hn, cn) = rnn(input, (h0, c0))


In [ ]:
output.shape

In [ ]:
class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()
        self.hidden_size = 64
        self.input_size = 3
        self.lstm = nn.LSTM(input_size=self.input_size, hidden_size=self.hidden_size, num_layers=1, batch_first=True)
        self.linear = nn.Linear(self.hidden_size, 1)

    def forward(self, x):
        # x has shape (batch_size, sequence_length, input_size)
        batch_size = x.size(0)
        h0 = torch.zeros(1, batch_size, self.hidden_size)
        c0 = torch.zeros(1, batch_size, self.hidden_size)
        out, _ = self.lstm(x, (h0, c0))
        out = self.linear(out[:, -1, :])
        return out.squeeze()


In [ ]:
model = RNN()

# Generate example input tensor
input_tensor = torch.randn(32, 20, 3)

# Pass input tensor through model to get predictions
predictions = model(input_tensor)
predictions.shape

## Testing a basic LSTM implementation

In [ ]:
def data_to_num(df):

    agent_choice_dict = {"Paper": 0, "Rock": 1, "Scissors": 2}
    result_dict = {"W": 1, "L": -1, "D": 0}

    df["agents_choice"] = df["agents_choice"].apply(lambda x: agent_choice_dict[x])
    df["result"] = df["result"].apply(lambda x: result_dict[x])

    return df

In [ ]:
# data = torch.Tensor()
# t = torch.Tensor()


agent1_df = pd.read_csv(DATA_PATH + 'agent1_df.csv', index_col=0)
agent2_df = pd.read_csv(DATA_PATH + 'agent2_df.csv',index_col=0)
agent3_df = pd.read_csv(DATA_PATH + 'agent3_df.csv',index_col=0)

print("Data before conversion:")
display(agent1_df.head(5))

agent1_df = data_to_num(agent1_df)
agent2_df = data_to_num(agent2_df)
agent3_df = data_to_num(agent3_df)

target1 = agent1_df.pop('result')
target2 = agent2_df.pop('result')
target3 = agent3_df.pop('result')

print("Data after conversion:")
display(agent1_df.head(5))

print("Target after conversion:")
display(target1.head(5))

In [ ]:
def get_sequences(df, seq_len=5):

    data = df.values # agent_choice
    
    data = np.array(data) == np.arange(3)

    input_seq = []
    target_seq = []

    for i in range(len(data) - seq_len):
        input_seq.append(data[i:i+seq_len])
        target_seq.append(data[i+seq_len, 0])

    return np.array(input_seq), np.array(target_seq)

input_seq, target_seq = get_sequences(agent1_df, seq_len=5)
print("Training Sequences: 1995, Window Size: 5, Number of Features: 1")
print(f"Input data shape: {input_seq.shape}")
print(f"Target data shape: {target_seq.shape}")
display(input_seq[:3], target_seq)

In [ ]:
# num columns in dataset
input_size = 3
# num hidden layers
hidden_size = 10
# rock, paper, scissors
output_size = 3

model = RNN()
["r", "p", "s"]
criterion = CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
type(input_seq)

In [ ]:
epochs = 100
data = torch.from_numpy(input_seq.astype("float32"))
t = torch.from_numpy(target_seq.astype("float32"))
batch_size = 64
num_baches = len(input_seq) // batch_size

for epoch in range(epochs):
    running_loss = 0
    for i in range(num_baches):
        inputs, targets = data[i*batch_size: (i+1)*batch_size], t[i*batch_size: (i+1)*batch_size]
        optimizer.zero_grad()
        output = model(inputs)
        loss = criterion(output, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch: {epoch}, Loss: {running_loss/num_baches}")

In [ ]:
rnn = nn.LSTM(10, 20, 2)
input = torch.randn(5, 3, 10)
h0 = torch.randn(2, 3, 20)
c0 = torch.randn(2, 3, 20)
output, (hn, cn) = rnn(input, (h0, c0))

print(output.shape)